Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train GenSen Model with Distributed Pytorch on AML
In this tutorial, you will train a GenSen model with PyTorch on AML using distributed training across a GPU cluster. This could also be a generic guideline to train models using GPU cluster.

Regarding **AzureML**, please refer to:
* [Quickstart notebook](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)
* [Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)

## 1. Global Settings

In [1]:
# set the environment path to find NLP
import sys
sys.path.append("../../../")
import time
import os
import papermill as pm
import pandas as pd
import shutil

import azureml as aml
import azureml.train.hyperdrive as hd
from azureml.telemetry import set_diagnostics_collection

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Azure ML SDK Version: 1.0.33
Pandas version: 0.24.2



Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


We assume that an AzureML [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) has already been created. For instructions on how to do this, see [here](README.md).

### 1.1 Initialize workspace

We create a workspace object using the configurations specified in `aml_config/config.json` that you created in the prerequisite step.

In [4]:
# AzureML workspace info. Note, will look up "./aml_config/config.json" first, then fall back to using this
SUBSCRIPTION_ID = '<subscription-id>'
RESOURCE_GROUP  = '<resource-group>'
WORKSPACE_NAME  = '<workspace-name>'

# Connect to a workspace
try:
    ws = aml.core.Workspace.from_config()
except aml.exceptions.UserErrorException:
    try:
        ws = aml.core.Workspace(
            subscription_id=SUBSCRIPTION_ID,
            resource_group=RESOURCE_GROUP,
            workspace_name=WORKSPACE_NAME
        )
        ws.write_config()
    except aml.exceptions.AuthenticationException:
        ws = None

if ws is None:
    raise ValueError(
        """Cannot access the AzureML workspace w/ the config info provided.
        Please check if you entered the correct id, group name and workspace name"""
    )
else:
    print("AzureML workspace name: ", ws.name)

AzureML workspace name:  MAIDAPNLP


## 2. Create or attach an existing remote compute target
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Specifically, the below code creates an `STANDARD_NC6` GPU cluster that autoscales from `0` to `4` nodes. 

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace, this code will skip the creation process and loads the cluster directly.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

VM_SIZE = 'STANDARD_NC6'
VM_PRIORITY = 'lowpriority'
MAX_NODES = 4

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                           max_nodes=MAX_NODES)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-05-02T21:01:04.496000+00:00', 'errors': None, 'creationTime': '2019-04-17T17:21:26.968570+00:00', 'modifiedTime': '2019-04-17T17:27:28.740980+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT7200S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Access to a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

`project_folder` contains all the code you want to submit to AmlCompute to run. The size of the folder can not exceed 300Mb. In GenSen model, it loads large pre-trained embedding files to the model. Thus, we need to save large files in datastore and only uploads code to `project_folder`.

In [ ]:
import os

# Change the path to where your model code locates.
# To download some of the data required to train a GenSen model in the original paper, run:
# https://github.com/Maluuba/gensen/blob/master/get_data.sh.
project_folder = '../../../models/advanced/gensen/amlcode/'
os.makedirs(project_folder, exist_ok=True)

## Access to Datastore
To download some of the data required to train a GenSen model, run the bash file [here](https://github.com/Maluuba/gensen/blob/master/get_data.sh). Make sure to upload all the large files to azure file share. You can access to datastore by using `ds.as_mount()`.

In [ ]:
from azureml.core import Datastore
ds = Datastore.register_azure_file_share(workspace=ws,
                                        datastore_name= 'GenSen',
                                        file_share_name='azureml-filestore-09b72610-7938-4ed2-86a2-5004896b12d9',
                                        account_name='maidapnlp0056795534',
                                        account_key='8LtGFZErNlvI6fSrgODqCxJCckkVgq3AL/5S/8ma7Re7xUHgWrNRCfTFnP/QDhF7KDY6ScAORsUpSm7ziog5/Q==')

Upload the contents of the data directory to the path ./data on the above datastore.

In [ ]:
# Upload files from local.
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

In [ ]:
ds.as_mount()
# We can fetch the datastore path by: os.environ['AZUREML_DATAREFERENCE_datarefname'].
# path_on_datastore = 'data/'
# ds_data = ds.path(path_on_datastore)
# print(ds_data)

## Train model on the remote compute
Now that we have the AmlCompute ready to go, let's run our distributed training job.

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training of GENSEN is already provided for you at `train.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

However, if you would like to use Azure ML's [metric logging](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#logging) capabilities, you will have to add a small amount of Azure ML logic inside your training script. In this example, at each logging interval, we will log the loss for that minibatch to our Azure ML run.

To do so, in `train.py`, we will first access the Azure ML `Run` object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Later within the script, we log the loss metric to our run:
```Python
run.log('loss', loss.item())
```

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial.

In [ ]:
from azureml.core import Experiment, get_run

experiment_name = 'pytorch-gensen'
experiment = Experiment(ws, name=experiment_name)


### Create a PyTorch estimator
The Azure ML SDK's PyTorch estimator enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-pytorch).

In [ ]:
from azureml.train.dnn import PyTorch
from azureml.train.estimator import Estimator

script_params = {
    '--config': 'example_config.json',
    '--data_folder': ds.as_mount()}

estimator = PyTorch(source_directory=project_folder,
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    node_count=4,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True,
                    conda_packages=['scikit-learn=0.20.3']
                   )

The above code specifies that we will run our training script on `4` nodes, with one worker per node. In order to execute a distributed run using GPU, you must provide the argument `use_gpu=True`. To execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, PyTorch, Horovod and their dependencies will be installed for you. If you are the first time to create a experiment, it may take longer to set up conda environments under `.azureml/conda_dependencies.yml`. After the first run, it will use the existing conda environments and directly run the code. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters. The more required packages are stored in `.azureml/conda_dependencies.yml` file.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(estimator)
print(run)

## Cancel the job.
It's better to cancel the job manually to make sure you does not waste resources.

In [ ]:
# Cancel the job with id.
# job_id = "pytorch-gensen_1555533596_d9cc75fe"
# run = get_run(experiment, job_id)

# Cancel jobs.
run.cancel()

### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes. You can see that the widget automatically plots and visualizes the loss metric that we logged to the Azure ML run.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
run.wait_for_completion(show_output=True) # this provides a verbose log